<a href="https://colab.research.google.com/github/abdullah-binmadhi/Abdullah-s-stuff/blob/main/BigData_MapReduce_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  # Read the CSV file into a pandas DataFrame
  try:
    df = pd.read_csv(fn)
    print("\nDataFrame head:")
    display(df.head())
  except Exception as e:
    print(f"Error reading file {fn}: {e}")

Saving Warehouse_and_Retail_Sales.csv to Warehouse_and_Retail_Sales.csv
User uploaded file "Warehouse_and_Retail_Sales.csv" with length 11350598 bytes

DataFrame head:


,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.0,1.0,0.0
1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.0,1.0,0.0
2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.0,8.0,0.0
3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.0,2.0,0.0
4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.0,4.0,0.0


In [3]:
display(df.head(10))

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.0,1.0,0.0
1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.0,1.0,0.0
2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.0,8.0,0.0
3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.0,2.0,0.0
4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.0,4.0,0.0
5,2017,4,REPUBLIC NATIONAL DISTRIBUTING CO,101680,MANISCHEWITZ CREAM WH CONCORD - 1.5L,WINE,0.0,1.0,0.0
6,2017,4,ROYAL WINE CORP,101753,BARKAN CLASSIC PET SYR - 750ML,WINE,0.0,1.0,0.0
7,2017,4,JIM BEAM BRANDS CO,10197,KNOB CREEK BOURBON 9YR - 100P - 1.75L,LIQUOR,0.0,32.0,0.0
8,2017,4,STE MICHELLE WINE ESTATES,101974,CH ST MICH P/GRIS - 750ML,WINE,0.0,26.0,0.0
9,2017,4,MONSIEUR TOUTON SELECTION,102083,CH DE LA CHESNAIE MUSCADET - 750ML,WINE,0.0,1.0,0.0


In [4]:
import pandas as pd

# Load the dataset
df = pd.read_csv("Warehouse_and_Retail_Sales.csv")

# Fill missing sales data with 0
df["RETAIL SALES"] = df["RETAIL SALES"].fillna(0)
df["WAREHOUSE SALES"] = df["WAREHOUSE SALES"].fillna(0)

# Calculate TOTAL SALES column
df["TOTAL SALES"] = df["RETAIL SALES"] + df["WAREHOUSE SALES"]


In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum

# Start Spark session
spark = SparkSession.builder \
    .appName("Sales Analysis") \
    .getOrCreate()

# Load dataset
df = spark.read.csv("Warehouse_and_Retail_Sales.csv", header=True, inferSchema=True)

# Clean sales columns and add total sales
df = df.fillna(0, subset=["RETAIL SALES", "WAREHOUSE SALES"])
df = df.withColumn("TOTAL SALES", col("RETAIL SALES") + col("WAREHOUSE SALES"))


In [15]:
top_products = df.groupBy("ITEM CODE", "ITEM DESCRIPTION") \
   .agg(_sum("TOTAL SALES").alias("TOTAL SALES")) \
   .orderBy(col("TOTAL SALES").desc()) \
   .limit(10)


top_products.show(truncate=False)

+---------+----------------------------------+------------------+
|ITEM CODE|ITEM DESCRIPTION                  |TOTAL SALES       |
+---------+----------------------------------+------------------+
|23445    |CORONA EXTRA LOOSE NR - 12OZ      |122552.37000000001|
|96750    |CORONA EXTRA 2/12 NR - 12OZ       |91522.17          |
|23886    |HEINEKEN LOOSE NR - 12OZ          |73191.79000000001 |
|90590    |MILLER LITE 30PK CAN - 12OZ       |61459.93000000001 |
|96970    |HEINEKEN 2/12 NR - 12OZ           |61120.479999999996|
|96741    |CORONA EXTRA 4/6 NR - 12OZ        |58808.21          |
|90468    |BUD LIGHT 30PK CAN                |45319.04          |
|96083    |HEINEKEN 4/6 NR - 12OZ            |45032.44          |
|25883    |MODELO ESPECIAL 24 LOOSE NR - 12OZ|40233.88          |
|23314    |CORONA EXTRA 18PK NR - 12OZ       |39213.11          |
+---------+----------------------------------+------------------+



In [10]:
supplier_sales = df.groupBy("SUPPLIER") \
    .agg(_sum("TOTAL SALES").alias("TOTAL SALES")) \
    .orderBy(col("TOTAL SALES").desc())

supplier_sales.show(truncate=False)


+---------------------------------+------------------+
|SUPPLIER                         |TOTAL SALES       |
+---------------------------------+------------------+
|MILLER BREWING COMPANY           |607646.0400000002 |
|CROWN IMPORTS                    |606532.5300000003 |
|ANHEUSER BUSCH INC               |536415.3300000002 |
|HEINEKEN USA                     |339735.76000000007|
|E & J GALLO WINERY               |143050.62000000008|
|DIAGEO NORTH AMERICA INC         |111909.24000000005|
|CONSTELLATION BRANDS             |99441.26999999996 |
|BOSTON BEER CORPORATION          |64194.15999999999 |
|YUENGLING BREWERY                |63433.670000000006|
|FLYING DOG BREWERY LLLP          |57528.29999999999 |
|PABST BREWING CO                 |54146.01999999999 |
|THE WINE GROUP                   |52361.930000000066|
|JIM BEAM BRANDS CO               |42085.239999999976|
|REPUBLIC NATIONAL DISTRIBUTING CO|40874.340000000084|
|LEGENDS LTD                      |39735.31999999998 |
|A VINTNER

In [11]:
type_sales = df.groupBy("ITEM TYPE") \
    .agg(_sum("TOTAL SALES").alias("TOTAL SALES")) \
    .orderBy(col("TOTAL SALES").desc())

type_sales.show(truncate=False)


+------------+------------------+
|ITEM TYPE   |TOTAL SALES       |
+------------+------------------+
|BEER        |2647380.43        |
|WINE        |746409.8899999887 |
|LIQUOR      |343021.1699999997 |
|KEGS        |43558.0           |
|NON-ALCOHOL |16766.69          |
|STR_SUPPLIES|995.98            |
|NULL        |1.0               |
|REF         |-6472.659999999999|
|DUNNAGE     |-45331.0          |
+------------+------------------+



In [12]:
monthly_sales = df.groupBy("YEAR", "MONTH") \
    .agg(_sum("TOTAL SALES").alias("TOTAL SALES")) \
    .orderBy("YEAR", "MONTH")

monthly_sales.show(truncate=False)


+----+-----+------------------+
|YEAR|MONTH|TOTAL SALES       |
+----+-----+------------------+
|2017|4    |0.0               |
|2017|5    |462102.2899999971 |
|2017|6    |476748.08999999485|
|2017|8    |469298.4699999975 |
|2017|9    |395893.12999999797|
|2017|10   |392951.8699999994 |
|2017|11   |437621.84999999637|
|2017|12   |438591.70999999944|
|2018|1    |330309.4400000005 |
|2018|2    |342812.6499999996 |
+----+-----+------------------+



In [13]:
retail_total = df.agg(_sum("RETAIL SALES").alias("Retail Total")).collect()[0][0]
warehouse_total = df.agg(_sum("WAREHOUSE SALES").alias("Warehouse Total")).collect()[0][0]

print(f"Retail = {int(retail_total)} units")
print(f"Warehouse = {int(warehouse_total)} units")


Retail = 842398 units
Warehouse = 2903930 units


In [14]:
threshold = 10

low_selling = df.groupBy("ITEM CODE", "ITEM DESCRIPTION") \
    .agg(_sum("TOTAL SALES").alias("TOTAL SALES")) \
    .filter(col("TOTAL SALES") < threshold) \
    .orderBy("TOTAL SALES")

low_selling.show(truncate=False)


+---------+--------------------------------------------+-----------+
|ITEM CODE|ITEM DESCRIPTION                            |TOTAL SALES|
+---------+--------------------------------------------+-----------+
|175      |EMPTY 1/2 KEG (30.00)                       |-34514.0   |
|205      |EMPTY 1/6 KEG (30.00)                       |-8248.0    |
|BC       |BEER CREDIT                                 |-6022.0    |
|264      |EMPTY 1/4 KEG (30.00)                       |-2513.0    |
|WC       |WINE CREDIT                                 |-732.0     |
|1279     |EMPTY WINE KEG - KEGS                       |-56.0      |
|31157    |DOGFISH HEAD FALL ON ME 1/2KG               |-2.0       |
|94722    |SAM ADAMS HOPSCAPE 1/2 KG                   |-2.0       |
|312627   |TWO OCEANS SHZ                 26936 - 750ML|-1.0       |
|28780    |FLYING DOG NITRO CP COFFEE PORTER 1/2KG     |-1.0       |
|340897   |AUBERT HYDE CHARD 14 - 750ML                |-1.0       |
|346644   |FUSE CAB 13 - 750ML    